In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

from pprint import pprint as pprint
from re import sub
from collections import namedtuple
from urllib2 import urlopen
from bs4 import BeautifulSoup
from datetime import datetime

%matplotlib inline

In [2]:
url = "https://nethouseprices.com/house-prices/london?page=1"
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')
# TO-DO: segment HTML downloading into chunks, and see if HTML can be saved to disk, and scraped by BeautifulSoup from disk.
# Also determine how much of the data to use as a dataset, and turn all your processing code into functions, so you can do
# this processing across multiple URLs to get one dataset you can model on.
# The next step is to do k-Means Clustering and get dummies, then run Random Forest Regressor, and write RESTful API.

# url_list = ["https://nethouseprices.com/house-prices/london?page={}".format(i) for i in xrange(1,12484)]
# html_list = [urlopen(url) for url in url_list]

In [3]:
# Identified from element inspections via the Chrome developer console.
addresses = soup.find_all("strong", class_="street-details-head-row")
prices = soup.find_all("strong", class_="street-details-price-row")
details = soup.find_all("div", class_="street-details-row")

In [4]:
sale_dates_rows = soup.find_all("tr", class_="sold_price_row")
sale_date_strings = [i.findChildren('td')[-1].text for i in sale_dates_rows]
def parse_date(date_string):
    ds = date_string
    dsl = ds.split(" ")
    return " ".join(["".join([char if not char.isalpha() else "" for char in dsl[0]]), " ".join(dsl[1:])])

cleaned_sale_date_strings = [parse_date(i) for i in sale_date_strings]
# print cleaned_sale_date_strings
sale_dates = [datetime.strptime(i, "%d %B %Y") for i in cleaned_sale_date_strings]
# sale_dates

In [5]:
# Our scraped data series are set below.
addr = [i.find("a").string.replace(u"\xa0", " ") for i in addresses]
pxs = [float(i.string.replace(u"\xa3", "").replace(u",", "")) for i in prices]
property_characteristics = [[i.strip() for i in categories.string.split(",")] for categories in details]
flat_type = [i[0] for i in property_characteristics]
lease_type = [i[1] for i in property_characteristics]
build_status = [i[2] for i in property_characteristics]

In [6]:
geodata_urls = ["https://nominatim.openstreetmap.org/search?q=\"{}\"&format=json".format(i.replace(" ", "%20")) for i in addr]
    
def get_geodata_object(openstreetmap_api_url):
    url = openstreetmap_api_url
    json_response_text = BeautifulSoup(urlopen(url), "lxml").text
    return json.loads(json_response_text)

def convert_json_to_named_tuple(json_):
    """This is solely for our convenience when referencing JSON response attributes in dataset creation"""
    return json.loads(json_, object_hook=lambda dict_: namedtuple('X', dict_.keys())(*dict_.values()))

In [7]:
json_search_results = [get_geodata_object(url) for url in geodata_urls]
top_search_results = [result if len(result) == 0 else result[0] for result in json_search_results]
# for i in top_search_results:
#     pprint(i)

In [8]:
# We now convert these top search results back to JSON to make named tuples for ease of referencing in dataseries creation.
# top_search_results_as_strings = ['[{}]'.format(str(i)) if type(i) == dict else '{}'.format(i) for i in top_search_results]
top_search_results_as_json = [json.dumps(i).replace("class", "category").replace("type", "subcategory").replace("osm_subcategory", "osm_type") for i in top_search_results]
geodata_json = [convert_json_to_named_tuple(result) for result in top_search_results_as_json]

In [9]:
def load_geodata_attributes(geodata_obj):
    try:
        return (
                geodata_obj.category,
                geodata_obj.subcategory,
                float(geodata_obj.importance),
                float(geodata_obj.lon),
                float(geodata_obj.lat))
    except AttributeError: # this handles the case where our JSON loader did not find a JSON response from the API URL.
        return (np.nan, np.nan, np.nan, np.nan, np.nan)

#TO-DO: add data parsed from display_name about borough as alternative to k Mean Clustering, to data frame.
# json_search_results = [get_geodata_object(url) for url in geodata_urls]
geodata = [load_geodata_attributes(result_named_tuple) for result_named_tuple in geodata_json]

category = [i[0] for i in geodata]
subcategory = [i[1] for i in geodata]
importance = [i[2] for i in geodata]
longitude = [i[3] for i in geodata]
latitude = [i[4] for i in geodata]

In [10]:
# print geodata

In [11]:
# We construct our dataset.
variables = [addr, pxs, sale_dates, flat_type, lease_type, build_status, category, subcategory, importance, longitude, latitude]
series_names = ["addresses",
                "prices",
                "sale_dates",
                "flat_type",
                "lease_type",
                "build_status",
                "category",
                "subcategory",
                "importance",
                "longitude",
                "latitude"]

# Check all series are the same length.
if len(pxs) == sum([len(dataseries) for dataseries in variables])/len(variables):
    # Setup dictionary for dataframe.
    dataset = {series_name : series for series_name, series in zip(series_names, variables)} 

In [12]:
# Beware some erroneous lon-lat data due to multiple search results and improper parsing of JSON response.
# NB: ignore the above, this was fixed. Now have to find a way of removing bad search data, say data 1SD away from mean lat lon.

dataset_frame = pd.DataFrame(dataset)
dataset_frame

,addresses,build_status,category,flat_type,importance,latitude,lease_type,longitude,prices,sale_dates,subcategory
0,"Flat 14 Buchanan House, 7 Troubridge Square, L...",Newbuild,NaN,Flat,NaN,NaN,Leasehold,NaN,517275.0,2018-09-26,NaN
1,"Flat 26 Buchanan House, 7 Troubridge Square, L...",Newbuild,NaN,Flat,NaN,NaN,Leasehold,NaN,527175.0,2018-09-26,NaN
2,"91 Dames Road, London, E7 0DW",Non-Newbuild,highway,Terraced,0.610,51.553902,Freehold,0.023391,420000.0,2018-09-25,secondary
3,"82 Ramsay Road, London, E7 9EW",Non-Newbuild,highway,Terraced,0.510,51.556909,Freehold,0.017585,420000.0,2018-09-25,residential
4,"21 Mandela Street, London, SW9 6EL",Non-Newbuild,highway,Terraced,0.520,51.478533,Freehold,-0.110912,560000.0,2018-09-25,residential
5,"115 Murray Road, London, W5 4DB",Non-Newbuild,highway,Terraced,0.510,51.493612,Leasehold,-0.313211,495000.0,2018-09-24,residential
6,"55 Briar Avenue, London, SW16 3AB",Non-Newbuild,highway,Semi Detached,0.610,51.417612,Freehold,-0.119799,535000.0,2018-09-24,residential
7,"26 Carlingford Road, London, N15 3EH",Non-Newbuild,highway,Terraced,0.610,51.589426,Freehold,-0.097470,500000.0,2018-09-24,residential
8,"30 Darwin Road, London, W5 4BD",Non-Newbuild,highway,Terraced,0.510,51.494709,Freehold,-0.305955,820000.0,2018-09-24,residential
9,"Flat 5 Peace Court, 8 Swynford Gardens, London...",Non-Newbuild,NaN,Flat,NaN,NaN,Leasehold,NaN,260000.0,2018-09-24,NaN


In [13]:
# We add a new parameter, which we will use to exclude anomalous lat lon coordinates from bad Open Street Map API data.
    
dataset_frame["latitude_z_score"] = (dataset_frame.latitude - dataset_frame.latitude.mean()) / dataset_frame.latitude.std()
dataset_frame["longitude_z_score"] = (dataset_frame.longitude - dataset_frame.longitude.mean()) / dataset_frame.longitude.std()
dataset_frame

,addresses,build_status,category,flat_type,importance,latitude,lease_type,longitude,prices,sale_dates,subcategory,latitude_z_score,longitude_z_score
0,"Flat 14 Buchanan House, 7 Troubridge Square, L...",Newbuild,NaN,Flat,NaN,NaN,Leasehold,NaN,517275.0,2018-09-26,NaN,NaN,NaN
1,"Flat 26 Buchanan House, 7 Troubridge Square, L...",Newbuild,NaN,Flat,NaN,NaN,Leasehold,NaN,527175.0,2018-09-26,NaN,NaN,NaN
2,"91 Dames Road, London, E7 0DW",Non-Newbuild,highway,Terraced,0.610,51.553902,Freehold,0.023391,420000.0,2018-09-25,secondary,0.196889,0.192977
3,"82 Ramsay Road, London, E7 9EW",Non-Newbuild,highway,Terraced,0.510,51.556909,Freehold,0.017585,420000.0,2018-09-25,residential,0.198578,0.192528
4,"21 Mandela Street, London, SW9 6EL",Non-Newbuild,highway,Terraced,0.520,51.478533,Freehold,-0.110912,560000.0,2018-09-25,residential,0.154565,0.182583
5,"115 Murray Road, London, W5 4DB",Non-Newbuild,highway,Terraced,0.510,51.493612,Leasehold,-0.313211,495000.0,2018-09-24,residential,0.163033,0.166928
6,"55 Briar Avenue, London, SW16 3AB",Non-Newbuild,highway,Semi Detached,0.610,51.417612,Freehold,-0.119799,535000.0,2018-09-24,residential,0.120355,0.181896
7,"26 Carlingford Road, London, N15 3EH",Non-Newbuild,highway,Terraced,0.610,51.589426,Freehold,-0.097470,500000.0,2018-09-24,residential,0.216838,0.183624
8,"30 Darwin Road, London, W5 4BD",Non-Newbuild,highway,Terraced,0.510,51.494709,Freehold,-0.305955,820000.0,2018-09-24,residential,0.163649,0.167489
9,"Flat 5 Peace Court, 8 Swynford Gardens, London...",Non-Newbuild,NaN,Flat,NaN,NaN,Leasehold,NaN,260000.0,2018-09-24,NaN,NaN,NaN


In [14]:
# We create new columns which introduce NaN data where the lat lon absolute Z scores exceed 3, indicating likely bad data.
# By bad data, we mean assuming a normal distribution, the lat lon coordinates are statistically significantly different
# from the average coordinates of all other datapoints. Given all the houses should be in London, this is a good filter.

dataset_frame["lat_z_score_mask"] = dataset_frame.latitude_z_score.where(dataset_frame.latitude_z_score.abs() < 3)
dataset_frame["lon_z_score_mask"] = dataset_frame.longitude_z_score.where(dataset_frame.longitude_z_score.abs() < 3)
dataset_frame

,addresses,build_status,category,flat_type,importance,latitude,lease_type,longitude,prices,sale_dates,subcategory,latitude_z_score,longitude_z_score,lat_z_score_mask,lon_z_score_mask
0,"Flat 14 Buchanan House, 7 Troubridge Square, L...",Newbuild,NaN,Flat,NaN,NaN,Leasehold,NaN,517275.0,2018-09-26,NaN,NaN,NaN,NaN,NaN
1,"Flat 26 Buchanan House, 7 Troubridge Square, L...",Newbuild,NaN,Flat,NaN,NaN,Leasehold,NaN,527175.0,2018-09-26,NaN,NaN,NaN,NaN,NaN
2,"91 Dames Road, London, E7 0DW",Non-Newbuild,highway,Terraced,0.610,51.553902,Freehold,0.023391,420000.0,2018-09-25,secondary,0.196889,0.192977,0.196889,0.192977
3,"82 Ramsay Road, London, E7 9EW",Non-Newbuild,highway,Terraced,0.510,51.556909,Freehold,0.017585,420000.0,2018-09-25,residential,0.198578,0.192528,0.198578,0.192528
4,"21 Mandela Street, London, SW9 6EL",Non-Newbuild,highway,Terraced,0.520,51.478533,Freehold,-0.110912,560000.0,2018-09-25,residential,0.154565,0.182583,0.154565,0.182583
5,"115 Murray Road, London, W5 4DB",Non-Newbuild,highway,Terraced,0.510,51.493612,Leasehold,-0.313211,495000.0,2018-09-24,residential,0.163033,0.166928,0.163033,0.166928
6,"55 Briar Avenue, London, SW16 3AB",Non-Newbuild,highway,Semi Detached,0.610,51.417612,Freehold,-0.119799,535000.0,2018-09-24,residential,0.120355,0.181896,0.120355,0.181896
7,"26 Carlingford Road, London, N15 3EH",Non-Newbuild,highway,Terraced,0.610,51.589426,Freehold,-0.097470,500000.0,2018-09-24,residential,0.216838,0.183624,0.216838,0.183624
8,"30 Darwin Road, London, W5 4BD",Non-Newbuild,highway,Terraced,0.510,51.494709,Freehold,-0.305955,820000.0,2018-09-24,residential,0.163649,0.167489,0.163649,0.167489
9,"Flat 5 Peace Court, 8 Swynford Gardens, London...",Non-Newbuild,NaN,Flat,NaN,NaN,Leasehold,NaN,260000.0,2018-09-24,NaN,NaN,NaN,NaN,NaN


In [15]:
# We will also create dummy variables for all our non-numerical dataseries.
build_status_dummies = pd.get_dummies(dataset_frame.build_status)
flat_type_dummies = pd.get_dummies(dataset_frame.flat_type)
lease_type_dummies = pd.get_dummies(dataset_frame.lease_type)
category_dummies = pd.get_dummies(dataset_frame.category)
subcategory_dummies = pd.get_dummies(dataset_frame.subcategory)


dataframes_set = [dataset_frame, build_status_dummies, flat_type_dummies, lease_type_dummies, category_dummies, subcategory_dummies]
# for dataframe in dataframes_set:
#     dataframe.reset_index(drop=True)
    
dataset_frame = pd.concat(dataframes_set, axis=1)
# dataset_frame = pd.concat([dataset_frame, pd.get_dummies(dataset_frame.lease_type)], axis=1)
dataset_frame

,addresses,build_status,category,flat_type,importance,latitude,lease_type,longitude,prices,sale_dates,...,place,shop,convenience,house,primary,residential,secondary,tertiary,uncategoryified,yes
0,"Flat 14 Buchanan House, 7 Troubridge Square, L...",Newbuild,NaN,Flat,NaN,NaN,Leasehold,NaN,517275.0,2018-09-26,...,0,0,0,0,0,0,0,0,0,0
1,"Flat 26 Buchanan House, 7 Troubridge Square, L...",Newbuild,NaN,Flat,NaN,NaN,Leasehold,NaN,527175.0,2018-09-26,...,0,0,0,0,0,0,0,0,0,0
2,"91 Dames Road, London, E7 0DW",Non-Newbuild,highway,Terraced,0.610,51.553902,Freehold,0.023391,420000.0,2018-09-25,...,0,0,0,0,0,0,1,0,0,0
3,"82 Ramsay Road, London, E7 9EW",Non-Newbuild,highway,Terraced,0.510,51.556909,Freehold,0.017585,420000.0,2018-09-25,...,0,0,0,0,0,1,0,0,0,0
4,"21 Mandela Street, London, SW9 6EL",Non-Newbuild,highway,Terraced,0.520,51.478533,Freehold,-0.110912,560000.0,2018-09-25,...,0,0,0,0,0,1,0,0,0,0
5,"115 Murray Road, London, W5 4DB",Non-Newbuild,highway,Terraced,0.510,51.493612,Leasehold,-0.313211,495000.0,2018-09-24,...,0,0,0,0,0,1,0,0,0,0
6,"55 Briar Avenue, London, SW16 3AB",Non-Newbuild,highway,Semi Detached,0.610,51.417612,Freehold,-0.119799,535000.0,2018-09-24,...,0,0,0,0,0,1,0,0,0,0
7,"26 Carlingford Road, London, N15 3EH",Non-Newbuild,highway,Terraced,0.610,51.589426,Freehold,-0.097470,500000.0,2018-09-24,...,0,0,0,0,0,1,0,0,0,0
8,"30 Darwin Road, London, W5 4BD",Non-Newbuild,highway,Terraced,0.510,51.494709,Freehold,-0.305955,820000.0,2018-09-24,...,0,0,0,0,0,1,0,0,0,0
9,"Flat 5 Peace Court, 8 Swynford Gardens, London...",Non-Newbuild,NaN,Flat,NaN,NaN,Leasehold,NaN,260000.0,2018-09-24,...,0,0,0,0,0,0,0,0,0,0


In [16]:
dataset_frame = dataset_frame.dropna()
cols_to_drop = ['build_status',
                'flat_type',
                'lease_type',
                'category',
                'subcategory',
                'latitude_z_score',
                'longitude_z_score',
                'lat_z_score_mask',
                'lon_z_score_mask']
final_dataset_frame = dataset_frame.drop(cols_to_drop, axis=1)
final_dataset_frame = final_dataset_frame.reset_index(drop=True)
final_dataset_frame

,addresses,importance,latitude,longitude,prices,sale_dates,Newbuild,Non-Newbuild,Detached,Flat,...,place,shop,convenience,house,primary,residential,secondary,tertiary,uncategoryified,yes
0,"91 Dames Road, London, E7 0DW",0.610,51.553902,0.023391,420000.0,2018-09-25,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
1,"82 Ramsay Road, London, E7 9EW",0.510,51.556909,0.017585,420000.0,2018-09-25,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,"21 Mandela Street, London, SW9 6EL",0.520,51.478533,-0.110912,560000.0,2018-09-25,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,"115 Murray Road, London, W5 4DB",0.510,51.493612,-0.313211,495000.0,2018-09-24,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,"55 Briar Avenue, London, SW16 3AB",0.610,51.417612,-0.119799,535000.0,2018-09-24,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
5,"26 Carlingford Road, London, N15 3EH",0.610,51.589426,-0.097470,500000.0,2018-09-24,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
6,"30 Darwin Road, London, W5 4BD",0.510,51.494709,-0.305955,820000.0,2018-09-24,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
7,"Flat 2 59 Dalston Lane, London, E8 2NG",0.620,51.546280,-0.069922,795000.0,2018-09-24,0,1,0,1,...,0,0,0,0,1,0,0,0,0,0
8,"5 Griffins Close, London, N21 2EW",0.610,51.633648,-0.084601,920000.0,2018-09-24,0,1,1,0,...,0,0,0,0,0,1,0,0,0,0
9,"56 Chatham Road, Walthamstow, London, E17 6EU",0.410,51.594836,0.015356,660000.0,2018-09-21,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
